In [40]:
import os
from math import exp
from typing import Dict, List

import tiktoken
from openai import OpenAI

GPT - Cross Encoder

In [4]:
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [8]:
labels = ["Yes", "No"]
MODEL_NAME = "gpt-4o-mini"
tokenizer = tiktoken.encoding_for_model(MODEL_NAME)
print(f"Tokenizer for {MODEL_NAME}: {tokenizer.name}")

Tokenizer for gpt-4o-mini: o200k_base


In [15]:
label_ids = [tokenizer.encode(label) for label in labels]
print(f"Label IDs: {label_ids}")
logit_bias = {id[0]: 1 for id in label_ids if len(id) == 1}
print(f"Logit bias: {logit_bias}")

Label IDs: [[13022], [3160]]
Logit bias: {13022: 1, 3160: 1}


In [43]:
ECOMMERCE_RANKING_PROMPT = '''
You are an Assistant responsible for helping detect whether the retrieved product is relevant to the query. For a given input, you need to output a single token: "Yes" or "No" indicating the retrieved product is relevant to the query.

Query: Younique setting powder
Product: 
"""
Younique Touch Behold Translucent Setting Powder

Touch Behold Translucent Setting Powder. Younique’s Touch Behold Translucent Setting Powder effortlessly locks and loads your look so you’re ready to take on the world. Use as the finishing touch to help keep makeup in place, or wear directly on skin for a softening, matte look.

product category: Beauty & Personal Care
"""
Relevant: Yes

Query: white musk hand cream
Product: 
"""
Braided Hair Clips for Women Girls, Sparkling Crystal Stone Braided Hair Clips Barrette with 3 Small Clips, Triple Hair Clips with Rhinestones for Sectioning,4PCS (4pcs-Type A)

product category: Beauty & Personal Care
"""
Relevant: No

Query: HP Pavilion dm4 replacement battery
Product: 
"""
ATC 11.1V 6-Cell Replacement Laptop Battery for HP Pavilion dm4-1062nr Pavilion dm4-1063cl Pavilion dm4-1063he Pavilion dm4-1065dx Pavilion dm4-1070ee Pavilion dm4-1070ef

product category: Electronics
"""
Relevant: Yes

Query: Cushionaire cork sandals
Product: 
"""
CUSHIONAIRE Women's Lane Cozy Cork footbed Sandal with Faux fur lining and +Comfort

Women's Cushionaire comfort Cork footbed sandal with Faux Fur lining. Stay cool with comfy sandals that will give you comfort throughout your day.

product category: Clothing Shoes & Jewelry
"""
Relevant: Yes

Query: under cabinet LED light
Product: 
"""
Christmas Snowflake Projector Lights Outdoor Led Snowfall Show with Remote Control Waterproof Landscape Decorative Lighting for Christmas Holiday Party Wedding Garden Patio

product category: Tools & Home Improvement
"""
Relevant: No 

Query: {query}
Product: 
"""
{product_text}
"""
Relevant:
'''

In [36]:
def product_relevance(
    client: OpenAI,
    prompt: str,
    model: str = MODEL_NAME,
    temperature: float = 0.0,
    logprobs: bool = True,
    logit_bias: Dict[int, int] = None,
    max_tokens: int = 1,
):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": prompt},
        ],
        temperature=temperature,
        logprobs=logprobs,
        logit_bias=logit_bias,
        max_tokens=max_tokens,
    )

    return response

In [37]:
query = ""
product_text = ""

In [38]:
responses = product_relevance(
    client,
    ECOMMERCE_RANKING_PROMPT.format(query=query, product_text=product_text),
    logit_bias=logit_bias,
)

In [41]:
label = responses.choices[0].message.content
logprob = responses.choices[0].logprobs.content[0].logprob
print(f"Label: {label} with logprob: {logprob} - probability: {exp(logprob)}")

Label: No with logprob: -3.1737043e-06 - probability: 0.9999968263007362


## Llama - Cross Encoder